In [0]:
!wget -O FSDKaggle2018.audio_train.zip https://zenodo.org/record/2552860/files/FSDKaggle2018.audio_train.zip?download=1
!wget -O FSDKaggle2018.audio_test.zip https://zenodo.org/record/2552860/files/FSDKaggle2018.audio_test.zip?download=1
!wget -O FSDKaggle2018.meta.zip https://zenodo.org/record/2552860/files/FSDKaggle2018.meta.zip?download=1
!unzip \*.zip

In [0]:
!mkdir data submission
%mv FSDKaggle2018.audio_train data/train
%mv FSDKaggle2018.audio_test data/test
%mv -v FSDKaggle2018.meta/* submission/
%cd data/train/
!ls -F |grep -v / | wc -l
%cd ../test/
!ls -F |grep -v / | wc -l
%cd ../..
!rm FSDKaggle2018.audio_train.zip FSDKaggle2018.audio_test.zip FSDKaggle2018.meta.zip
!rm -R FSDKaggle2018.meta

In [0]:
!wget -O summary_feats_funcs.py https://drive.google.com/open?id=1BEXV2BN9il2_O1L-uGPYGnTJKnOZjzJz

In [0]:
!find . -name "*.wav" -type f -delete

In [0]:
import librosa
import os
import re
import sys

import pandas as pd
import numpy as np

from sklearn import  preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC

import tensorflow as tf

from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU, Bidirectional
from tensorflow.keras.layers import Conv1D, Conv2D, GlobalMaxPooling1D
from tensorflow.keras.layers import Activation, Flatten, Dropout
from tensorflow.keras.models import Sequential,model_from_json
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import keras 

from keras import regularizers
from keras import backend as K

from keras.activations import relu, softmax
from keras.layers import Input, InputLayer, Dense,Flatten,Dropout,Activation, BatchNormalization, Conv1D, GlobalAveragePooling1D, MaxPool1D, GlobalMaxPooling1D, LSTM
from keras.layers.merge import add
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

from joblib import Parallel, delayed

from tqdm import tqdm

import pickle

In [0]:
sys.path.append('/content')

In [0]:
!pip install pushbullet.py

In [0]:

df_train_og = pd.read_csv("/content/submission/train_post_competition.csv")

## 1. Pre-process

In [0]:
"""Pre-process"""

sampling_rate = 44100

#train = pd.read_csv("data/train.csv")
train = df_train_og
samp_subm = df_train_og[["fname"]].copy()
samp_subm['label'] = ""

# Ignoring the empty wavs
samp_subm['toremove'] = 0
samp_subm.loc[samp_subm.fname.isin([
    '0b0427e2.wav', '6ea0099f.wav', 'b39975f5.wav'
]), 'toremove'] = 1

# print('Train...')
# os.makedirs('data/audio_train_trim/', exist_ok=True)
# for filename in tqdm(train.fname.values):
#     x, sr = librosa.load('data/audio_train/' + filename, sampling_rate)
#     x = librosa.effects.trim(x)[0]
#     np.save('data/audio_train_trim/' + filename + '.npy', x)

print('Test...')
os.makedirs('data/audio_test_trim/', exist_ok=True)
for filename in tqdm(samp_subm.loc[lambda x: x.toremove == 0, :].fname.values):
    x, sr = librosa.load('data/train/' + filename, sampling_rate)
    x = librosa.effects.trim(x)[0]
    np.save('data/audio_test_trim/' + filename + '.npy', x)

## 2. Compute Log-Mel

In [0]:
"""Compute Log Mel-Spectrograms"""
# calculate log mel: https://datascience.stackexchange.com/questions/27634/how-to-convert-a-mel-spectrogram-to-log-scaled-mel-spectrogram
# Paper: https://arxiv.org/pdf/1608.04363.pdf



sampling_rate = 44100

train = df_train_og
samp_subm = df_train_og[["fname"]].copy()
samp_subm['label'] = ""

# Ignoring the empty wavs
# samp_subm['toremove'] = 0
# samp_subm.loc[samp_subm.fname.isin([
#     '0b0427e2.wav', '6ea0099f.wav', 'b39975f5.wav'
# ]), 'toremove'] = 1


def compute_melspec(filename, indir, outdir):
    wav = np.load(indir + filename + '.npy')
    wav = librosa.resample(wav, 44100, 22050)
    melspec = librosa.feature.melspectrogram(wav,
                                             sr=22050,
                                             n_fft=1764,
                                             hop_length=220,
                                             n_mels=64)
    logmel = librosa.core.power_to_db(melspec)
    np.save(outdir + filename + '.npy', logmel)


# print('Train...')
# os.makedirs('/data/mel_spec_train', exist_ok=True)
# for x in tqdm(train.fname.values):
#     compute_melspec(x, 'data/audio_train_trim/', 'data/mel_spec_train/')

print('Test...')
os.makedirs('data/mel_spec_test/', exist_ok=True)
for x in tqdm(samp_subm.loc[lambda x: x.toremove == 0, :].fname.values):
    compute_melspec(x, 'data/audio_test_trim/', 'data/mel_spec_test/')

## 3. Compute Summary metrics of various spectral and time based features

In [0]:

# number of cores to use
num_cores = 1

# print('Train...')
# train = pd.read_csv('data/train.csv')

# train_feats = Parallel(n_jobs=num_cores)(
#     delayed(all_feats)('data/audio_train_trim/' + x + '.npy')
#     for x in tqdm(train.fname.values))

# train_feats_df = pd.DataFrame(np.vstack(train_feats))
# train_feats_df['fname'] = pd.Series(train.fname.values, index=train_feats_df.index)
# train_feats_df.to_pickle('data/train_tab_feats.pkl')


print('Test...')
# samp_subm = pd.read_csv("data/sample_submission.csv")
# samp_subm['toremove'] = 0
# samp_subm.loc[samp_subm.fname.isin([
#     '0b0427e2.wav', '6ea0099f.wav', 'b39975f5.wav'
# ]), 'toremove'] = 1


test_feats = Parallel(n_jobs=num_cores)(
    delayed(all_feats)('data/audio_test_trim/' + x + '.npy')
    for x in tqdm(samp_subm
                  .loc[lambda x: x.toremove == 0, :]
                  .fname.values))

test_feats_df = pd.DataFrame(np.vstack(test_feats))
test_feats_df['fname'] = pd.Series(samp_subm.loc[lambda x: x.toremove == 0, :].fname.values,
                                   index=test_feats_df.index)
test_feats_df.to_pickle('data/test_tab_feats.pkl')

## Get files from original

In [0]:
file_to_tag = pd.Series(df_train_og['label'].values,index=df_train_og['fname']).to_dict()

In [0]:
def getTag(x):
    return (file_to_tag[x])

In [0]:
# function Clickconnect() {
#   console.log('Working')
#   document
#     .querySelector('#top-toolbar > colab-connect-button')
#     .shadowRoot.querySelector('#connect')
#     .click()
# }

# setInterval(Clickconnect, 60000)

## Import Dataset

In [0]:
import pickle

pickle_in = open("train_tab_feats.pkl","rb")
df_train = pickle.load(pickle_in)

pickle_in = open("test_tab_feats.pkl","rb")
df_test = pickle.load(pickle_in)

In [0]:
total = pd.concat([df_train,df_test],ignore_index=True)

#### Need usable test file

In [0]:
#total['tag'] = total['fname'].apply(getTag)

In [0]:
df_train['tag'] = df_train['fname'].apply(getTag)

In [0]:
df_train_copy = df_train.drop(['fname','tag'], axis = 1)

## Reduce Dimensions and Make Train Val Sets

In [0]:
LDA = LinearDiscriminantAnalysis()
X = LDA.fit_transform(df_train_copy, df_train['tag'])

x_train, x_val, y_train, y_val = train_test_split(X,  df_train['tag'], shuffle = True, test_size = 0.2, random_state = 42)

encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.fit_transform(y_val)

## SGD Linear Model

In [0]:
SGD = SGDClassifier()
SGD.fit(x_train,y_train)
y_pred = SGD.predict(x_val)


In [0]:
new_index = list(encoder.classes_)
new_index.append('accuracy')
new_index.append('macro avg')
new_index.append('weighted avg')

In [0]:
report = classification_report(y_val, y_pred, output_dict=True)
df_sgd_first = pd.DataFrame(report).transpose()
df_sgd_first.index = new_index

In [0]:
df_sgd_first

## SVM

In [0]:
svm = SVC()
svm.fit(x_train,y_train)
y_pred = svm.predict(x_val)

In [0]:
new_index = list(encoder.classes_)
new_index.append('accuracy')
new_index.append('macro avg')
new_index.append('weighted avg')

In [0]:
report = classification_report(y_val, y_pred, output_dict=True)
df_svm_first = pd.DataFrame(report).transpose()
df_svm_first.index = new_index

In [0]:
df_svm_first

## Grid Search On SVM

In [0]:
def SVC_GridSearch(X, Y, X_test, Y_test):
    svc = SVC()
    parameters = {
        'C': (0.5,1,2),
        'kernel': ('rbf','linear','poly', 'sigmoid'),
        'shrinking': (True, False),
        'decision_function_shape': ('ovp','ovr'),
        

    }
    grid_search = GridSearchCV(svc, parameters, n_jobs=-1, verbose=0)
    grid_search.fit(X, Y)
    accuracy = grid_search.best_score_
    best_parameters = grid_search.best_estimator_.get_params()
    classifier = grid_search.best_estimator_
    y_pred = classifier.predict(X_test)
    test_accuracy = accuracy_score(y_pred, Y_test)
    return best_parameters, accuracy, test_accuracy

In [0]:
best_parameters, accuracy, test_accuracy = SVC_GridSearch(x_train, y_train, x_val, y_val)

In [0]:
bestSVM = SVC()
bestSVM.set_params(**best_parameters)


In [0]:
bestSVM.fit(x_train,y_train)
y_pred = bestSVM.predict(x_val)

In [0]:
new_index = list(encoder.classes_)
new_index.append('accuracy')
new_index.append('macro avg')
new_index.append('weighted avg')

In [0]:
report = classification_report(y_val, y_pred, output_dict=True)
df_svm = pd.DataFrame(report).transpose()
df_svm.index = new_index

In [0]:
df_svm

## Vanilla Neural Network

In [0]:
# Set the input and output sizes
input_size = 40 #NUMBER INPUTS HERE#
output_size = 41 #NUMBER OUTPUTS HERE#


#DEFINE HIDDEN LAYER SIZE
#CAN HAVE MULTIPLE DIFFERENT SIZED LAYERS IF NEEDED
#50 NICE START POINT FOR BEING TIME EFFICIENT BUT STILL RELATIVELY COMPLEX
hidden_layer_size = 100
  

#MODEL SPECIFICATIONS
model = Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    Dropout(0.2),
    Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    Dropout(0.2),
    Dense(hidden_layer_size, activation='relu'), # 3rd hidden layer
    Dropout(0.2),


    # POTENTIALLY MULTIPLE MORE LAYERS HERE #
    # NO SINGLE ACTIVATION NECESSARILY BEST (AT THIS STAGE I DO NOT FULLY UNDERSTAND DIFFERENCES, TRY DIFFERENT VARIATIONs)
    
    # FINAL LAYER MUST TAKE OUTPUT SIZE
    #FOR CLASSIFICATION PROBLEMS USE SOFTMAX AS ACTIVATION
    Dense(output_size, activation='softmax') # output layer
])


#COMPILE MODEL GIVING IT OPTIMIZER LOSS FUNCTION AND METRIC OF INTEREST
# MOST TIMES USE ADAM FOR OPTIMIZER (LOOK AT OTHERS THOUGH) 
# lOSS FUNCTION - MANY DIFFERENT VARIATIONS sparse_categorical_crossentropy IS BASICALLY MIN SUM OF SQUARES
# TO NOW I AM ONLY INTERESTED IN ACCURACY AT EACH LEVEL (HAVE NOT LOOKED AT OTHER OPTIONS`)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


###                            ### 
###                            ###
###          TRAINING          ###
###                            ###
###                            ###

# SET SIZE OF BATCHES (FOR SHUFFLING IN PARTS WHEN OVERALL SIZE TO BIG)
batch_size = 128

# SET MAXIMUM NUMBER OF EPOCHS (JUST SO DOESNT RUN ENDLESSLY)
max_epochs = 100

# SET EARLY STOPPING FUNCTION
# PATIENCE EQUAL 0 (DEFAULT) => STOPS AS SOON AS FOLLOWING EPOCH HAS REDUCED LOSS
# PATIENCE EQUAL N => STOPS AFTER N SUBSEQUENT INCREASING LOSSES
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)



###                            ### 
###                            ###
###         FIT MODEL          ###
###                            ###
###                            ###

model.fit(x_train, # train inputs
          y_train, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          callbacks=[early_stopping], # early stopping
          validation_data=(x_val, y_val), # validation data
          verbose = 1 # shows some information for each epoch so we can analyse
          )  

In [0]:
y_pred = model.predict(x_val)
y_pred = np.argmax(y_pred, axis=1)
vanilla_nn = classification_report(y_val, y_pred)
print(vanilla_nn)

In [0]:
from scipy.stats import skew
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

sr = 44100


def compute_summ_features(x):
    ans = np.hstack((
        np.mean(x, axis=1),
        np.std(x, axis=1),
        skew(x, axis=1),
        np.max(x, axis=1),
        np.min(x, axis=1)))
    return ans


def feat_set_1(x, stft):
    # Features mentioned in
    # - http://aqibsaeed.github.io/2016-09-03-urban-sound-classification-part-1/
    # - https://www.kaggle.com/amlanpraharaj/xgb-using-mfcc-opanichev-s-features-lb-0-811

    # Mel-scaled power spectrogram
    mels = librosa.feature.melspectrogram(x, sr=sr, S=stft)

    # Mel-frequency cepstral coefficients
    mfccs = librosa.feature.mfcc(y=x, sr=sr, S=stft, n_mfcc=40)

    # chorma-stft: Compute a chromagram from a waveform or power spectrogram
    chromas = librosa.feature.chroma_stft(S=stft, sr=sr)

    # spectral_contrast: Compute spectral contrast
    contrasts = librosa.feature.spectral_contrast(x, S=stft, sr=sr)

    # Compute roll-off frequency
    rolloffs = librosa.feature.spectral_rolloff(x, sr=sr, S=stft)

    # Compute the spectral centroid
    scentroids = librosa.feature.spectral_centroid(x, sr=sr, S=stft)

    # Compute p’th-order spectral bandwidth
    bandwidths = librosa.feature.spectral_bandwidth(x, sr=sr, S=stft)

    # tonnetz: Computes the tonal centroid features (tonnetz)
    tonnetzs = librosa.feature.tonnetz(y=librosa.effects.harmonic(x), sr=sr)

    # zero crossing rate
    zero_crossing_rates = librosa.feature.zero_crossing_rate(x)

    tmp = (mels, mfccs, chromas, contrasts,
           rolloffs, scentroids, bandwidths,
           tonnetzs, zero_crossing_rates)

    ans = np.hstack([
        compute_summ_features(x)
        for x in tmp
    ])

    return ans


# Features from https://www.kaggle.com/opanichev/lightgbm-baseline
def calc_part_features(data, n=2):
    ans = []
    for j, i in enumerate(range(0, len(data), len(data)//n)):
        if j == (n-1):
            i = len(data) - 1
        if j < n:
            ans.append(np.mean(data[i:i + len(data)//n]))
            ans.append(np.std(data[i:i + len(data)//n]))
            ans.append(np.min(data[i:i + len(data)//n]))
            ans.append(np.max(data[i:i + len(data)//n]))
    return ans


def feat_set_4(x):
    abs_data = np.abs(x)
    diff_data = np.diff(x)

    ans = []

    n = 1
    ans += calc_part_features(x, n=n)
    ans += calc_part_features(abs_data, n=n)
    ans += calc_part_features(diff_data, n=n)

    n = 2
    ans += calc_part_features(x, n=n)
    ans += calc_part_features(abs_data, n=n)
    ans += calc_part_features(diff_data, n=n)

    n = 3
    ans += calc_part_features(x, n=n)
    ans += calc_part_features(abs_data, n=n)
    ans += calc_part_features(diff_data, n=n)

    return np.array(ans)


# Features from https://www.kaggle.com/agehsbarg/audio-challenge-cnn-with-concatenated-inputs
def get_spectra_win(y, L, N):
    dft = np.fft.fft(y)
    fl = np.abs(dft)
    xf = np.arange(0.0, N/L, 1/L)
    return (xf, fl)


def get_spectra(signal, fs, M=1000, sM=500):

    N = signal.shape[0]
    ind = np.arange(100, N, M)

    spectra = []
    meanspectrum = np.repeat(0, M)

    for k in range(1, len(ind)):
        n1 = ind[k-1]
        n2 = ind[k]
        y = signal[n1:n2]
        L = (n2-n1)/fs
        N = n2-n1
        (xq, fq) = get_spectra_win(y, L, N)
        spectra.append(fq)

    spectra = pd.DataFrame(spectra)
    meanspectrum = spectra.apply(lambda x: np.log(1+np.mean(x)), axis=0)
    stdspectrum = spectra.apply(lambda x: np.log(1+np.std(x)), axis=0)

    meanspectrum = meanspectrum[0:sM]
    stdspectrum = stdspectrum[0:sM]

    return (meanspectrum, stdspectrum)


def get_width(w):
    if np.sum(w) == 0:
        return [0, 0, 0]
    else:
        z = np.diff(np.where(np.insert(np.append(w, 0), 0, 0) == 0))-1
        z = z[z > 0]
    return [np.log(1+np.mean(z)),
            np.log(1+np.std(z)),
            np.log(1+np.max(z)),
            len(z)]


def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[N:] - cumsum[:-N]) / float(N)


# predictors related to peaks
def num_peaks(x):
    x = np.array(x[0:len(x)])
    n10 = np.sum(x > 0.10*np.max(x))
    n20 = np.sum(x > 0.20*np.max(x))
    n50 = np.sum(x > 0.50*np.max(x))
    n90 = np.sum(x > 0.90*np.max(x))
    n99 = np.sum(x > 0.99*np.max(x))
    lead_min = np.min(np.where(x == np.max(x)))
    w10 = get_width(1*(x > 0.10*np.max(x)))
    w20 = get_width(1*(x > 0.20*np.max(x)))
    w50 = get_width(1*(x > 0.50*np.max(x)))
    w90 = get_width(1*(x > 0.90*np.max(x)))
    w99 = get_width(1*(x > 0.99*np.max(x)))
    W = w10+w20+w50+w90+w99

    f_sc = np.sum(np.arange(0, len(x))*(x*x)/np.sum(x*x))

    i1 = np.where(x < 0.10*np.max(x))[0]
    if i1.size == 0:
        lincoef_w = [0, 0, 0]
    else:
        a1 = i1[i1 < lead_min]
        a2 = i1[i1 > lead_min]

        if a1.size == 0:
            i1_left = 0
        else:
            i1_left = np.max(i1[i1 < lead_min])
        if a2.size == 0:
            i1_right = 0
        else:
            i1_right = np.min(i1[i1 > lead_min])

        lead_min_width = i1_right - i1_left
        if (lead_min_width > 2):
            poly_w = PolynomialFeatures(degree=2, include_bias=False)
            f_ind_w = poly_w.fit_transform(
                np.arange(i1_left, i1_right, 1).reshape(-1, 1))
            clf_w = linear_model.LinearRegression()
            linmodel_w = clf_w.fit(f_ind_w, np.array(x[i1_left:i1_right]))
            lincoef_w = list(linmodel_w.coef_)+[linmodel_w.intercept_]
        else:
            lincoef_w = [0, 0, 0]

    S = np.sum(x)
    S_n = np.sum(x)/len(x)
    S2 = np.sqrt(np.sum(x*x))
    S2_n = np.sqrt(np.sum(x*x))/len(x)
    integrals = [S, S_n, S2, S2_n]

    poly = PolynomialFeatures(degree=2, include_bias=False)
    f_ind = poly.fit_transform(np.arange(0, len(x)).reshape(-1, 1))
    clf = linear_model.LinearRegression()
    linmodel = clf.fit(f_ind, x)
    lincoef_spectrum = list(linmodel.coef_)+[linmodel.intercept_]

    high_freq_sum_50 = np.sum(x[0:50] >= 0.5*np.max(x))
    high_freq_sum_90 = np.sum(x[0:50] >= 0.9*np.max(x))

    r = [f_sc, n10, n20, n50, n90, n99,
         lead_min, high_freq_sum_50, high_freq_sum_90] \
        + W + lincoef_spectrum + integrals + lincoef_w
    return r


def runningMeanFast(x, N=20):
    return np.convolve(x, np.ones((N,))/N)[(N-1):]


def feat_set_2(x):
    rawsignal = x
    rawsignal_sq = rawsignal*rawsignal
    silenced = []
    sound = []
    attack = []
    for wd in [2000]:
        rawsignal_sq_rm = running_mean(rawsignal_sq, wd)
        w1 = 1*(rawsignal_sq_rm < 0.01*np.max(rawsignal_sq_rm))
        silenced = silenced + get_width(w1)
        w2 = 1*(rawsignal_sq_rm < 0.05*np.max(rawsignal_sq_rm))
        silenced = silenced + get_width(w2)
        w3 = 1*(rawsignal_sq_rm > 0.05*np.max(rawsignal_sq_rm))
        sound = sound + get_width(w3)
        w4 = 1*(rawsignal_sq_rm > 0.25*np.max(rawsignal_sq_rm))
        sound = sound + get_width(w4)
        time_to_attack = np.min(np.where(
            rawsignal_sq_rm > 0.99*np.max(rawsignal_sq_rm)))
        time_rel = np.where(rawsignal_sq_rm < 0.2*np.max(rawsignal_sq_rm))[0]
        if (time_rel.size == 0):
            time_to_relax = len(rawsignal_sq_rm)
        elif (time_rel[time_rel > time_to_attack].size == 0):
            time_to_relax = len(rawsignal_sq_rm)
        else:
            time_to_relax = np.min(time_rel[time_rel > time_to_attack])
        attack.append(np.log(1+time_to_attack))
        attack.append(np.log(1+time_to_relax))

    lr = len(rawsignal)
    zerocross_tot = np.log(
        1 + np.sum(
            np.array(
                rawsignal[0:(lr-1)]
            ) * np.array(rawsignal[1:lr]) <= 0))
    zerocross_prop = np.sum(
        np.array(
            rawsignal[0:(lr-1)]) * np.array(rawsignal[1:lr]) <= 0) / lr
    return np.array(sound + attack + [zerocross_tot, zerocross_prop])


def feat_set_3(x):
    (m, sd) = get_spectra(x, sr, 2000, 1000)
    ans1 = np.array(num_peaks(m))
    ans2 = (lambda x: x[np.arange(0, len(x), 40)])(np.array(runningMeanFast(m)))
    return np.concatenate((ans1, ans2))


def all_feats(filename):
    x = np.load(filename)
    stft = np.abs(librosa.stft(x))
    out1 = feat_set_1(x, stft=stft)
    out2 = feat_set_2(x)
    out3 = feat_set_3(x)
    out4 = feat_set_4(x)

    assert out1.shape[0] == 985
    assert out2.shape[0] == 12
    assert out3.shape[0] == 64
    assert out4.shape[0] == 72

    return np.concatenate((
        out1, out2, out3, out4
    ))
